In [10]:
import http.client
import json
from settings import API_KEY
import time
from pandas.io.json import json_normalize
import pandas as pd
import  csv

## Funções

In [ ]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	return(json.loads(data.decode("    utf-8")))

In [ ]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
        
		try:
			credits_data = get_json(conn, url)
		except Exception as e:
			if (credits_data['status_code'] == 25):
				time.sleep(10)                
				credits_data = get_json(conn, url)        
        
		df = json_normalize(credits_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

In [ ]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    for idx,movie_id in enumerate(ids,40):

        movie_item = pd.DataFrame()
        movie_item = movie_credits_df[movie_credits_df['id']==int(movie_id)].reset_index()
        print("%s of %s" % (idx,len(ids)))

        print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]: # ESSA LINHA ESTÁ DANDO PROBLEMA DE INDEX            
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

## Testes

In [3]:
movie_credits_df = pd.read_csv('movie_credits_df.csv')

In [4]:
movie_credits_df.shape

(147507, 4)

In [7]:
movie_credits_df = movie_credits_df.drop_duplicates('id', keep='first')

In [8]:
movie_credits_df.shape

(115909, 4)

In [11]:
with open('out_sample.csv', 'r') as f:
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist]

In [12]:
len(ids)

156